In [20]:
import psycopg2 as pg
import pandas.io.sql as psql
import yaml
import json
from pathlib import Path

with open('../utils/secrets.json', 'r') as f:
    config = json.load(f)
query = """
select
  id, 
  array_to_string(array(select jsonb_array_elements_text(hashtags::jsonb)), ' ') as tags
from
  metabase.posts
"""
connection = pg.connect(f"host={config['host']} dbname={config['database']} user={config['user']} password={config['password']}")
dataframe = psql.read_sql(query, connection)
print(dataframe.head())

/tmp/ipykernel_514446/4163039823.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = psql.read_sql(query, connection)


       id                     tags
0  153391                         
1  153400                         
2  153405                         
3  153408  Ukraine nuclearfootball
4  153409                         


In [21]:
dataframe_filtered = dataframe[dataframe['tags'].str.len()>0]

In [23]:
dataframe_filtered['tags'] = dataframe_filtered['tags'].apply(lambda x: x.lower().strip())

/tmp/ipykernel_514446/1361170881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_filtered['tags'] = dataframe_filtered['tags'].apply(lambda x: x.lower().strip())


In [25]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english', min_df=2, max_features=5000)

In [27]:
tfidf_wm = tfidfvectorizer.fit_transform(dataframe_filtered['tags'].to_list())
tfidf_tokens = tfidfvectorizer.get_feature_names_out()

In [ ]:
from sklearn.manifold import TSNE
embeddings = TSNE(n_components=20, learning_rate='auto', init='random', perplexity=3).fit_transform(tfidf_wm.toarray())